In [119]:
import torch
import torch.nn as nn
import numpy as np 
import torch.nn.functional as F 

import visdom
vis = visdom.Visdom()

Setting up a new session...


In [154]:
class fc_model(nn.Module):
    def __init__(self):
        super(fc_model, self).__init__()
        self.linear1 = nn.Linear(28*28, 256)
        self.linear2 = nn.Linear(256, 64)
        self.linear3 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

class cnn_model(nn.Module):
    def __init__(self):
        super(cnn_model, self).__init__()
        self.conv1 = nn.Conv2d(1, 28, 3, 1)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(28, 14, 3, 1)
        self.pool2 = nn.MaxPool2d(2)
        self.linear = nn.Linear(350, 10)

    def forward(self, x):
        x = self.pool1(self.conv1(x))
        x = self.pool2(self.conv2(x))
        x = x.view((x.size(0), -1))
        x = self.linear(x)
        return x

        
class rnn_model(nn.Module):
    def __init__(self):
        super(rnn_model, self).__init__()
        self.num_layers = 1
        self.hidden_size = 64
        self.rnn1 = nn.LSTM(28, self.hidden_size, 1, batch_first=True)
        self.linear = nn.Linear(self.hidden_size, 10)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        x, (hn,cn) = self.rnn1(x, (h0, c0))
        x = self.linear(x[:, -1, :])
        return x


models = [fc_model(), cnn_model(), rnn_model()]        

In [157]:
#--------------------------------------------- file read
x_train = np.load("data/mnist_train.npy")
x_test  = np.load("data/mnist_test.npy")
y_train = np.load("data/mnist_train_target.npy")
y_test  = np.load("data/mnist_test_target.npy")
#--------------------------------------------- numpy to tensor
x_train  = torch.from_numpy(x_train).float()       #long으로 하면 loss 계산할 때 에러
x_test   = torch.from_numpy(x_test).float()
y_train  = torch.from_numpy(y_train).long()        #float으로 하면 loss 계산할 때 에러  
y_test   = torch.from_numpy(y_test).long()

#--------------------------------------------- data to dataset
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
test_dataset  = torch.utils.data.TensorDataset(x_test,  y_test)

#--------------------------------------------- dataset to dataloader 
train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=256,
                                          shuffle=True,
                                          num_workers=2)
                                    
test_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=256,
                                          shuffle=True,
                                          num_workers=2)                                 
                                    

In [143]:
model = models[0]
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

EPOCH = 1
for t in range(EPOCH): # EPOCH
    for i, (sample, target) in enumerate(train_loader): #BATCH
        sample = sample.view((sample.size(0), -1))
        y = model(sample)

        loss = criterion(y, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 100 == 99:
            print(t, loss.item())
        vis.line(X=[i], Y=[loss.item()], win="loss",name="FC" , 
                    update='append',opts=dict(showlegend=True))

0 1.976778268814087
0 1.2632031440734863
0 0.9096055626869202
0 0.6970781683921814


In [144]:
model = models[1]
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

EPOCH = 1
for t in range(EPOCH): # EPOCH
    for i, (sample, target) in enumerate(train_loader): #BATCH
        sample = sample.unsqueeze(1)
        y = model(sample)

        loss = criterion(y, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 100 == 99:
            print(t, loss.item())
        vis.line(X=[i], Y=[loss.item()], win="loss",name="CNN" ,
                update='append', opts=dict(showlegend=True))

0 2.11556339263916
0 1.971940279006958
0 0.8097959756851196
0 0.7934888601303101


In [158]:
model = models[2]
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

EPOCH = 1
for t in range(EPOCH): # EPOCH
    for i, (sample, target) in enumerate(train_loader): #BATCH
        #print(sample.size())
        y = model(sample)
        loss = criterion(y, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 100 == 99:
            print(t, loss.item())        
        vis.line(X=[i], Y=[loss.item()], win="loss",name="RNN" ,
                update='append', opts=dict(showlegend=True))

0 2.323206663131714


KeyboardInterrupt: 

In [146]:
# Test

correct = 0
total = 0
with torch.no_grad(): 
    for data in test_loader:
        images, labels = data
        images = images.view(images.size()[0], -1)   
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

RuntimeError: input must have 3 dimensions, got 2

In [ ]:
print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}')